In [1]:
import pandas as pd
import os
from pathlib import Path
import re
import cv2
import hashlib

In [34]:
files = []
filenames = []
directory = './raw_data/'
for file in os.listdir(directory):
    if file.split('.')[-1] == 'csv':
        files.append(pd.read_csv(directory + file))
        filenames.append(directory + file)

In [35]:
files[4].head()

,image,id,choice,annotator,annotation_id
0,/data/upload/4356.jpeg,573,"{""""choices"""": [""""trunk"""", """"dirty""""]}",stasechka28@yandex.ru,708
1,/data/upload/4366.jpeg,576,"{""""choices"""": [""""interior"""", """"dirty""""]}",stasechka28@yandex.ru,705
2,/data/upload/4410.jpeg,617,"{""""choices"""": [""""trunk"""", """"dirty""""]}",stasechka28@yandex.ru,666
3,/data/upload/4468.jpeg,675,"{""""choices"""": [""""left"""", """"labeled"""", """"damage...",stasechka28@yandex.ru,609
4,/data/upload/4533.jpeg,753,"{""""choices"""": [""""interior"""", """"dirty""""]}",stasechka28@yandex.ru,544


In [36]:
df = pd.concat(files)

In [37]:
df.shape

(4265, 5)

### Исправление битого файла

In [29]:
strange_filename = filenames[4]
file = []
with open(strange_filename) as f:
    line = f.readline()
    file.append(line)
    while line:
        line = f.readline()
        if line.startswith('"'):
            line = line[1:-2]
        else:
            line = line[:-1]
            
        # Убрать лишние кавычки в середине строки
        pos = line.find(',"')
        if pos != -1:
            pos += 1
            line = line[:pos] + line[pos+1:]
            
            pos = line.rfind('",')
            line = line[:pos] + line[pos+1:]
            
        file.append(line + '\n')

In [30]:
file[1]

'/data/upload/4356.jpeg,573,"{""""choices"""": [""""trunk"""", """"dirty""""]}",stasechka28@yandex.ru,708\n'

In [31]:
new_filename = directory + 'reworked.csv'
with open(new_filename, 'w') as f:
    for line in file:
        f.write(line)

In [32]:
t = pd.read_csv(new_filename)

In [33]:
t.head()

,image,id,choice,annotator,annotation_id
0,/data/upload/4356.jpeg,573,"{""""choices"""": [""""trunk"""", """"dirty""""]}",stasechka28@yandex.ru,708
1,/data/upload/4366.jpeg,576,"{""""choices"""": [""""interior"""", """"dirty""""]}",stasechka28@yandex.ru,705
2,/data/upload/4410.jpeg,617,"{""""choices"""": [""""trunk"""", """"dirty""""]}",stasechka28@yandex.ru,666
3,/data/upload/4468.jpeg,675,"{""""choices"""": [""""left"""", """"labeled"""", """"damage...",stasechka28@yandex.ru,609
4,/data/upload/4533.jpeg,753,"{""""choices"""": [""""interior"""", """"dirty""""]}",stasechka28@yandex.ru,544


### Обработка

In [38]:
def clear_filename(s):
    """Очистить имя файла от пути и суффикса от label-studio."""
    s = Path(s).name
    pos = s.find('_')
    if pos != -1:
        end = s.rfind('.')
        s = s[:pos] + s[end:]
    return s

def get_number(s):
    """Вернуть номер изображения."""
    pos = s.find('.')
    s = s[:pos]
    return int(s)

In [39]:
df['image'] = df['image'].apply(clear_filename)
df['id'] = df['image'].apply(get_number)

df['choice'] = df['choice'].fillna('')
df['choice'] = df['choice'].apply(lambda x: x.lower())

df.drop(['annotator', 'annotation_id'], axis=1, inplace=True)

In [40]:
df.head()

,image,id,choice
0,4595.jpeg,4595,b_left
1,4606.jpeg,4606,"{""choices"": [""back"", ""labeled""]}"
2,4703.jpeg,4703,"{""choices"": [""f_left"", ""labeled""]}"
3,4744.jpeg,4744,"{""choices"": [""interior"", ""dirty""]}"
4,4768.jpeg,4768,"{""choices"": [""right"", ""dirty"", ""damaged""]}"


In [41]:
classes = {"front": 0,
           "back": 1,
           "left": 2,
           "right": 3,
           "f_left": 4, 
           "f_right": 5,
           "b_left": 6,
           "b_right": 7,
           "interior": 8,
           "trunk": 9,
           "panel": 10,
           "dirty": 11,
           "labeled": 12,
           "damaged": 13}

def expand_choice(row):
    """Преобразовать choice в широкий формат."""
    row = row.copy()
    if row['choice'] == '':
        return row
    
    s = re.sub(r'(choices)|[\{\}\":\[\] ]', '', row['choice'])
    arr = s.split(',')
    for label in arr:
        row[f'class_{classes[label]}'] = 1
    return row

In [42]:
for i in range(14):
    df[f'class_{i}'] = 0
    
df = df.apply(expand_choice, axis=1)

In [43]:
df.head()

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13
0,4595.jpeg,4595,b_left,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,4606.jpeg,4606,"{""choices"": [""back"", ""labeled""]}",0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,4703.jpeg,4703,"{""choices"": [""f_left"", ""labeled""]}",0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,4744.jpeg,4744,"{""choices"": [""interior"", ""dirty""]}",0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,4768.jpeg,4768,"{""choices"": [""right"", ""dirty"", ""damaged""]}",0,0,0,1,0,0,0,0,0,0,0,1,0,1


### Заполнение пропущенных объектов

In [44]:
files_from_dataset = set(df['image'])
files_from_folder = set(os.listdir('dataset'))
missed_files = files_from_folder - files_from_dataset
len(missed_files)

301

In [45]:
def make_missed_dataset(files, columns):
    """Собрать датасет из пропущенных файлов."""
    result = []
    for file in files:
        row = [
            file,
            get_number(file),
            ''
        ]
        row += [0] * 14
        result.append(row)
        
    result = pd.DataFrame(result, columns=columns)
    return result

In [46]:
missed_df = make_missed_dataset(missed_files, df.columns)

In [47]:
missed_df.head()

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13
0,4631.jpeg,4631,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2936.jpg,2936,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1416.jpg,1416,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2042.jpg,2042,,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2046.jpg,2046,,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
full_df = pd.concat([df, missed_df])
full_df = full_df.sort_values(by='id')
full_df.reset_index(drop=True, inplace=True)

In [49]:
full_df.duplicated().sum()

15

In [50]:
full_df.shape

(4566, 17)

In [51]:
full_df = full_df.drop_duplicates()
full_df.shape

(4551, 17)

In [52]:
full_df[full_df.duplicated(subset='image', keep=False)]

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13
1847,1847.jpg,1847,"{""choices"": [""right"", ""labeled""]}",0,0,0,1,0,0,0,0,0,0,0,0,1,0
1848,1847.jpg,1847,"{""choices"": [""right"", ""labeled"", ""damaged""]}",0,0,0,1,0,0,0,0,0,0,0,0,1,1


На картинке нет повреждений, удалим первую строку.

In [53]:
full_df.drop(1847, inplace=True)
full_df.shape

(4550, 17)

In [54]:
full_df.reset_index(drop=True, inplace=True)
full_df.head()

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13
0,0.jpg,0,panel,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1.jpg,1,right,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.jpg,2,interior,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3.jpg,3,panel,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4.jpg,4,interior,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [55]:
full_df.tail()

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13
4545,4940.jpg,4940,"{""choices"": [""back"", ""labeled""]}",0,1,0,0,0,0,0,0,0,0,0,0,1,0
4546,4941.jpg,4941,"{""choices"": [""b_left"", ""labeled""]}",0,0,0,0,0,0,1,0,0,0,0,0,1,0
4547,4942.jpg,4942,"{""choices"": [""left"", ""labeled""]}",0,0,1,0,0,0,0,0,0,0,0,0,1,0
4548,4943.jpg,4943,"{""choices"": [""f_left"", ""labeled""]}",0,0,0,0,1,0,0,0,0,0,0,0,1,0
4549,4944.jpg,4944,"{""choices"": [""interior"", ""dirty""]}",0,0,0,0,0,0,0,0,1,0,0,1,0,0


### Обогащение датасета

1. Город
2. Ширина и высота изображения
3. Среднее значение, стандартное отклонение по RGB

In [57]:
cities = pd.read_excel(directory + 'ids_2.xlsx')
cities.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,C:,Users,glebs,Documents,2. DataScience_Projects,Practice,6_P2_NVBS,3_Images,all,Астрахань,2021,Май,Н723МС30/10.jpg,NaN
1,1,C:,Users,glebs,Documents,2. DataScience_Projects,Practice,6_P2_NVBS,3_Images,all,Астрахань,2021,Май,Н723МС30/6.jpg,NaN
2,2,C:,Users,glebs,Documents,2. DataScience_Projects,Practice,6_P2_NVBS,3_Images,all,Астрахань,2021,Май,Н723МС30/9.jpg,NaN
3,3,C:,Users,glebs,Documents,2. DataScience_Projects,Practice,6_P2_NVBS,3_Images,all,Астрахань,2021,Март,Н723МС30/ФОТО 10.jpg,NaN
4,4,C:,Users,glebs,Documents,2. DataScience_Projects,Practice,6_P2_NVBS,3_Images,all,Астрахань,2021,Март,Н723МС30/ФОТО 9.jpg,NaN


In [58]:
full_df['city'] = full_df.merge(cities, how='left', left_on='id', right_on='Unnamed: 0').loc[:, 9]

In [59]:
full_df.head()

,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,class_11,class_12,class_13,city
0,0.jpg,0,panel,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Астрахань
1,1.jpg,1,right,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Астрахань
2,2.jpg,2,interior,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Астрахань
3,3.jpg,3,panel,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Астрахань
4,4.jpg,4,interior,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Астрахань


In [60]:
def get_image_params(row):
    row = row.copy()
    s = row['image']
    path = str(Path('./').joinpath('dataset', s))
    img = cv2.imread(path)
    row['height'], row['width'], _ = img.shape
    
    # среднее значение по каналам
    img = img.astype(float) / 255.
    row['r_mean'] = img[:, :, 0].mean()
    row['g_mean'] = img[:, :, 1].mean()
    row['b_mean'] = img[:, :, 2].mean()
    
    # стандартное отклонение по каналам
    row['r_std'] = img[:, :, 0].std()
    row['g_std'] = img[:, :, 1].std()
    row['b_std'] = img[:, :, 2].std()

    return row

In [61]:
%%time

full_df = full_df.apply(get_image_params, axis=1)
full_df.head()

CPU times: user 10min 12s, sys: 57.4 s, total: 11min 10s
Wall time: 12min 44s


,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,...,class_13,city,height,width,r_mean,g_mean,b_mean,r_std,g_std,b_std
0,0.jpg,0,panel,0,0,0,0,0,0,0,...,0,Астрахань,957,1280,0.664191,0.662805,0.649829,0.248083,0.248339,0.259096
1,1.jpg,1,right,0,0,0,1,0,0,0,...,0,Астрахань,957,1280,0.596717,0.605751,0.580831,0.255681,0.252227,0.258267
2,2.jpg,2,interior,0,0,0,0,0,0,0,...,0,Астрахань,957,1280,0.600527,0.595664,0.581935,0.316734,0.323528,0.332438
3,3.jpg,3,panel,0,0,0,0,0,0,0,...,0,Астрахань,960,1280,0.464392,0.478320,0.487675,0.189722,0.183444,0.178202
4,4.jpg,4,interior,0,0,0,0,0,0,0,...,0,Астрахань,960,1280,0.450628,0.462521,0.465585,0.209926,0.222312,0.228740


In [62]:
def md5(s):
    """https://stackoverflow.com/questions/3431825/generating-an-md5-checksum-of-a-file"""
    fname = str(Path('./').joinpath('dataset', s))
    
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [63]:
%%time

full_df['md5'] = full_df['image'].apply(md5)
full_df.head()

CPU times: user 3.76 s, sys: 800 ms, total: 4.57 s
Wall time: 4.66 s


,image,id,choice,class_0,class_1,class_2,class_3,class_4,class_5,class_6,...,city,height,width,r_mean,g_mean,b_mean,r_std,g_std,b_std,md5
0,0.jpg,0,panel,0,0,0,0,0,0,0,...,Астрахань,957,1280,0.664191,0.662805,0.649829,0.248083,0.248339,0.259096,5d4b35aaa33358623fe66153744bd01a
1,1.jpg,1,right,0,0,0,1,0,0,0,...,Астрахань,957,1280,0.596717,0.605751,0.580831,0.255681,0.252227,0.258267,9f98baecf558334f86e320d092a41ffa
2,2.jpg,2,interior,0,0,0,0,0,0,0,...,Астрахань,957,1280,0.600527,0.595664,0.581935,0.316734,0.323528,0.332438,947d2f3d3b7089285caacda49cad7438
3,3.jpg,3,panel,0,0,0,0,0,0,0,...,Астрахань,960,1280,0.464392,0.478320,0.487675,0.189722,0.183444,0.178202,f967b7f08a2946d59c3e18cd4f9e1e32
4,4.jpg,4,interior,0,0,0,0,0,0,0,...,Астрахань,960,1280,0.450628,0.462521,0.465585,0.209926,0.222312,0.228740,9cc3f11f235406683586dae8ed07af9b


In [64]:
full_df.duplicated(subset=['md5']).sum()

45

In [65]:
full_df.drop_duplicates(subset=['md5'], inplace=True)
full_df.shape

(4505, 27)

In [66]:
full_df.to_csv('nvbs_auto.csv', index=False)